<a href="https://colab.research.google.com/github/j1c4b/ColabStocks/blob/main/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: import bs4 frombeautiful soap and requests

!pip install beautifulsoup4 requests

import requests
from bs4 import BeautifulSoup


In [3]:
# prompt: create web scalping python code to get reporting  date, company, current price, rating from https://www.marketbeat.com/ratings/ using  bs4 from beautiful soap and requests

import requests
from bs4 import BeautifulSoup

def scrape_marketbeat():
    url = "https://www.marketbeat.com/ratings/"
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table containing the data
    table = soup.find("table", class_="table table-striped table-hover")  # Replace with actual class if different

    if not table:
        print("Table not found on page.")
        return []

    data = []
    rows = table.find_all("tr")
    for row in rows[1:]:  # Skip header row
      cols = row.find_all("td")
      if len(cols) >= 4:  # Check if there are at least 4 columns
          reporting_date = cols[0].text.strip()
          company = cols[1].text.strip()
          try:
              current_price = float(cols[2].text.strip().replace('$','')) # Convert price to float
          except ValueError:
              current_price = None # Handle cases where price might not be a number
          rating = cols[3].text.strip()

          data.append({
              "reporting_date": reporting_date,
              "company": company,
              "current_price": current_price,
              "rating": rating
          })
    return data

# Example usage
scraped_data = scrape_marketbeat()
for item in scraped_data:
  item


Table not found on page.


In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of MarketBeat's Analyst Ratings page
URL = "https://www.marketbeat.com/ratings/"

# Set headers to mimic a real browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# File path
file_path = "marketbeat_analyst_ratings.csv"

# Check if file exists and delete if it does
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Existing file '{file_path}' deleted.")

print(f"File location: {os.path.abspath(file_path)}")

# Request the page
response = requests.get(URL, headers=HEADERS)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table containing analyst ratings
    table = soup.find("table", class_="scroll-table sort-table")

    if table:
        data = []
        rows = table.find_all("tr")[1:]  # Skip the header row

        for row in rows:
            cols = row.find_all("td")

            # Check if the row has enough columns before accessing them
            if len(cols) >= 4:  # Make sure we have all the columns we need
                # Extract necessary details
            #    reporting_date = cols[0].text.strip()
            # Extract 'Symbol|Company' and split it into two parts
                symbol_company = cols[0].get("data-clean", "").strip()  # Get the attribute value
                symbol, company = symbol_company.split("|") if "|" in symbol_company else (symbol_company, "")

              #  symbol = cols[0].text.strip()
                action = cols[1].text.strip()
                current_price_price_change =cols[4].get("data-clean", "").strip()  # Get the attribute value
                current_price, price_change = current_price_price_change.split("|") if "|" in current_price_price_change else (current_price_price_change, "")

              #  cols[4].text.strip().replace('$','')
                rating = cols[6].text.strip()

                # Store the extracted data
               # data.append([reporting_date, company, current_price, rating])
                data.append([symbol, company, action, current_price, price_change, rating])

        # Convert to DataFrame for easy manipulation
       # df = pd.DataFrame(data, columns=["Reporting Date", "Company", "Current Price", "Rating"])\
        df = pd.DataFrame(data, columns=["Symbol", "Company", "Action", "Current Price", "Price Change", "Rating"])

        # Print the extracted data
        print(df)

        # Save to CSV (optional)
        df.to_csv("marketbeat_analyst_ratings.csv", index=False)
        print("\n✅ Data saved to 'marketbeat_analyst_ratings.csv'")

    else:
        print("❌ Failed to find the ratings table.")

else:
    print(f"❌ Request failed with status code {response.status_code}")

Existing file 'marketbeat_analyst_ratings.csv' deleted.
File location: /content/marketbeat_analyst_ratings.csv
    Symbol                  Company             Action Current Price  \
0     AAOI  Applied Optoelectronics      Reiterated by        $21.86   
1     AAPL                    Apple      Reiterated by       $212.21   
2     ABNB                   Airbnb   Target Raised by       $122.19   
3     ACIU                AC Immune      Reiterated by         $2.30   
4      ACN                Accenture  Target Lowered by       $316.63   
..     ...                      ...                ...           ...   
201    WAY                  Waystar       Initiated by        $36.10   
202   WEST          Westrock Coffee      Reiterated by         $6.78   
203   XPOF       Xponential Fitness  Target Lowered by         $7.54   
204   XPOF       Xponential Fitness  Target Lowered by         $7.54   
205   XPOF       Xponential Fitness      Downgraded by         $7.54   

    Price Change        

In [ ]:
# prompt: add code to above to send result file to gcp buskewt

import os
import uuid

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# --- Previous code ---

# --- New code to upload to GCS ---

# Make a unique bucket to which we'll upload the file.
# (GCS buckets are part of a single global namespace.)
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/mb
!gsutil mb gs://{bucket_name}

# Copy the file to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp marketbeat_analyst_ratings.csv gs://{bucket_name}/

# Finally, dump the contents of our newly copied file to make sure everything worked.
!gsutil cat gs://{bucket_name}/marketbeat_analyst_ratings.csv


In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of MarketBeat's Analyst Ratings page
URL = "https://www.marketbeat.com/ratings/"

# Set headers to mimic a real browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# File path
file_path = "marketbeat_analyst_ratings.csv"

# Check if file exists and delete if it does
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Existing file '{file_path}' deleted.")

print(f"File location: {os.path.abspath(file_path)}")

# Request the page
response = requests.get(URL, headers=HEADERS)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table containing analyst ratings
    table = soup.find("table", class_="scroll-table sort-table")

    if table:
        data = []
        rows = table.find_all("tr")[1:]  # Skip the header row

        for row in rows:
            cols = row.find_all("td")

            # Check if the row has enough columns before accessing them
            if len(cols) >= 4:  # Make sure we have all the columns we need
                # Extract necessary details
            #    reporting_date = cols[0].text.strip()
            # Extract 'Symbol|Company' and split it into two parts
                symbol_company = cols[0].get("data-clean", "").strip()  # Get the attribute value
                symbol, company = symbol_company.split("|") if "|" in symbol_company else (symbol_company, "")

              #  symbol = cols[0].text.strip()
                action = cols[1].text.strip()
                current_price_price_change =cols[4].get("data-clean", "").strip()  # Get the attribute value
                current_price, price_change = current_price_price_change.split("|") if "|" in current_price_price_change else (current_price_price_change, "")

              #  cols[4].text.strip().replace('$','')
                rating = cols[6].text.strip()

                # Store the extracted data
               # data.append([reporting_date, company, current_price, rating])
                data.append([symbol, company, action, current_price, price_change, rating])

        # Convert to DataFrame for easy manipulation
       # df = pd.DataFrame(data, columns=["Reporting Date", "Company", "Current Price", "Rating"])\
        df = pd.DataFrame(data, columns=["Symbol", "Company", "Action", "Current Price", "Price Change", "Rating"])

        # Print the extracted data
        print(df)

        # Save to CSV (optional)
        df.to_csv("marketbeat_analyst_ratings.csv", index=False)
        print("\n✅ Data saved to 'marketbeat_analyst_ratings.csv'")

    else:
        print("❌ Failed to find the ratings table.")

else:
    print(f"❌ Request failed with status code {response.status_code}")

Existing file 'marketbeat_analyst_ratings.csv' deleted.
File location: /content/marketbeat_analyst_ratings.csv
    Symbol                  Company             Action Current Price  \
0     AAOI  Applied Optoelectronics      Reiterated by        $22.15   
1     AAPL                    Apple      Reiterated by       $212.20   
2     ABNB                   Airbnb   Target Raised by       $122.00   
3     ACIU                AC Immune      Reiterated by         $2.29   
4      ACN                Accenture  Target Lowered by       $316.19   
..     ...                      ...                ...           ...   
201    WAY                  Waystar       Initiated by        $36.13   
202   WEST          Westrock Coffee      Reiterated by         $6.78   
203   XPOF       Xponential Fitness  Target Lowered by         $7.53   
204   XPOF       Xponential Fitness  Target Lowered by         $7.53   
205   XPOF       Xponential Fitness      Downgraded by         $7.53   

    Price Change        

In [ ]:
# prompt: modify above code to be run from ubuntu directory /home/user_name/projects/python_apps/stock_raiting

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import uuid
from google.colab import auth
import gspread
from google.auth import default

# Install necessary libraries if not already installed
!pip install beautifulsoup4 requests gspread oauth2client

# Authenticate with Google Drive/Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Specify the file path
file_path = "/home/user_name/projects/python_apps/stock_raiting/marketbeat_analyst_ratings.csv"


def scrape_marketbeat():
    # ... (rest of your scraping function code)


# Example usage
scraped_data = scrape_marketbeat()  # Assuming you have this function defined
# ... (rest of your code to process scraped_data)

# ... (rest of the code to scrape and save to csv)


# --- New code to upload to Google Sheets ---
try:
    # Open the spreadsheet (replace with your actual spreadsheet name or ID)
    spreadsheet_name = "Your Spreadsheet Name"  # Replace with the name of your spreadsheet
    sh = gc.open(spreadsheet_name)

    # Select the worksheet (replace with your actual worksheet name)
    worksheet_name = "Sheet1" # Replace with the name of your worksheet
    worksheet = sh.worksheet(worksheet_name)

    # Read the CSV data into a pandas DataFrame
    df = pd.read_csv(file_path)  # Now using the specified file path

    # Clear existing data in the worksheet (optional)
    worksheet.clear()

    # Update the worksheet with the DataFrame data
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

    print(f"✅ Data successfully uploaded to Google Sheet: {spreadsheet_name}, Worksheet: {worksheet_name}")

except gspread.exceptions.SpreadsheetNotFound:
    print(f"❌ Spreadsheet '{spreadsheet_name}' not found.")
except gspread.exceptions.WorksheetNotFound:
    print(f"❌ Worksheet '{worksheet_name}' not found in spreadsheet '{spreadsheet_name}'.")
except Exception as e:
    print(f"❌ An error occurred: {e}")


In [ ]:
# prompt: add code to send result file in email

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

def send_email(sender_email, sender_password, receiver_email, subject, body, filename):
    # Create a multipart message
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    # Open the file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = smtplib.SMTP_SSL("smtp.gmail.com", 465)
    context.login(sender_email, sender_password)
    context.sendmail(sender_email, receiver_email, text)
    context.quit()

# Example usage
sender_email = "your_email@gmail.com"  # Replace with your email
sender_password = "your_password"  # Replace with your email password
receiver_email = "recipient_email@example.com"  # Replace with recipient's email
subject = "Marketbeat Analyst Ratings"
body = "Please find the attached Marketbeat Analyst Ratings file."
filename = "marketbeat_analyst_ratings.csv"

send_email(sender_email, sender_password, receiver_email, subject, body, filename)
